# Lab - Classification

In this lab, we are going to build a classification module. When given an image of a handwritten digit like the one below, the model will be able to tell which digit is in the image.

<img src='test2.jpg'>

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random


# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split

In [27]:
path_to_dataset = r'digits_dataset'
target_img_size = (32, 32) # fix image size because classification algorithms THAT WE WILL USE HERE expect that

# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

## Part I - Feature Extraction

In this part, we are going to implement three functions. Each one will extract a different set of features from the image. The three sets are:

1. Histogram of the pixel values features (this is the histogram you know, but on the HSV channels)
2. Histogram of Gradients (HoG) features
3. Raw pixels (basically, not doing any feature extraction and just supplying the input image to the classifier)

In [28]:
def extract_hsv_histogram(img):
    """
    TODO
    1. Resize the image to target_img_size using cv2.resize
    2. Convert the image from BGR representation (cv2 is BGR not RGB) to HSV using cv2.cvtColor
    3. Acquire the histogram using the cv2.calcHist. Apply the functions on the 3 channels. For the bins 
        parameter pass (8, 8, 8). For the ranges parameter pass ([0, 180, 0, 256, 0, 256]). Name the histogram
        <hist>.
    """
    
    img = cv2.resize(img, target_img_size)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([img], [0, 1, 2], None, [8, 8, 8], [0, 180, 0, 256, 0, 256])

    if imutils.is_cv2():
        hist = cv2.normalize(hist)
    else:
        cv2.normalize(hist, hist)
    return hist.flatten()     

In [29]:
def extract_hog_features(img):
    """
    TODO
    You won't implement anything in this function. You just need to understand it 
    and understand its parameters (i.e win_size, cell_size, ... etc)
    """
    img = cv2.resize(img, target_img_size)
    win_size = (32, 32)
    cell_size = (4, 4)
    block_size_in_cells = (2, 2)
    
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0])
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten()

In [30]:
def extract_raw_pixels(img):
    """
    TODO
    The classification algorithms we are going to use expect the input to be a vector not a matrix. 
    This is because they are general purpose and don't work only on images.
    CNNs, on the other hand, expect matrices since they operate on images and exploit the 
    arrangement of pixels in the 2-D space.
    So, what we only need to do in this function is to resize and flatten the image.
    """
    img = cv2.resize(img, target_img_size)
    return img.flatten()


In [7]:
def extract_features(img, feature_set='hog'):
    """
    TODO
    Given either 'hsv_hist', 'hog', 'raw', call the respective function and return its output
    """
    if feature_set == 'hsv_hist':
        return extract_hsv_histogram(img)
    elif feature_set == 'hog':
        return extract_hog_features(img)
    elif feature_set == 'raw':
        return extract_raw_pixels(img)
    else:
        raise ValueError('Invalid feature set')

The following function will extract the features and the label of each image in our dataset and save it in RAM. We normally don't save datasets in RAM, but this dataset is small.

In [31]:
def load_dataset(feature_set='hog'):
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)

    for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'jpg':
            continue

        label = fn.split('.')[0]
        labels.append(label)

        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_features(img, feature_set))
        
        # show an update every 1,000 images
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels        

## Part II - Classification

In this part, we will test the classification performance of SVM, KNN, & NNs given our features.

In [32]:
# TODO understand the hyperparameters of each classifier
classifiers = {
    'SVM': svm.LinearSVC(random_state=random_seed),
    'KNN': KNeighborsClassifier(n_neighbors=7),
    'NN': MLPClassifier(solver='sgd', random_state=random_seed, hidden_layer_sizes=(1000,), max_iter=50, verbose=1)
}

In [33]:
# This function will test all our classifiers on a specific feature set
def run_experiment(feature_set):
    
    # Load dataset with extracted features
    print('Loading dataset. This will take time ...')
    features, labels = load_dataset(feature_set)
    print('Finished loading dataset.')
    
    # Since we don't want to know the performance of our classifier on images it has seen before
    # we are going to withhold some images that we will test the classifier on after training 
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    
    for model_name, model in classifiers.items():
        print('############## Training', model_name, "##############")
        # Train the model only on the training features
        model.fit(train_features, train_labels)
        
        # Test the model on images it hasn't seen before
        accuracy = model.score(test_features, test_labels)
        
        print(model_name, 'accuracy:', accuracy*100, '%')

Now, we see how each classifier and each feature set performs

In [34]:
run_experiment('hog')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 97.70833333333333
KNN accuracy ~ 96.52777777777779
NN accuracy ~ 93.95833333333333
"""

Loading dataset. This will take time ...
[INFO] processed 1000/7200
[INFO] processed 2000/7200
[INFO] processed 3000/7200
[INFO] processed 4000/7200
[INFO] processed 5000/7200
[INFO] processed 6000/7200
[INFO] processed 7000/7200
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 97.63888888888889 %
############## Training KNN ##############
KNN accuracy: 96.80555555555556 %
############## Training NN ##############
Iteration 1, loss = 2.16545631
Iteration 2, loss = 2.00208862
Iteration 3, loss = 1.84139771
Iteration 4, loss = 1.68711687
Iteration 5, loss = 1.53951903
Iteration 6, loss = 1.39941103
Iteration 7, loss = 1.26883443
Iteration 8, loss = 1.15010975
Iteration 9, loss = 1.04391852
Iteration 10, loss = 0.95021685
Iteration 11, loss = 0.86811862
Iteration 12, loss = 0.79676134
Iteration 13, loss = 0.73477835
Iteration 14, loss = 0.68111529
Iteration 15, loss = 0.63433920
Iteration 16, loss = 0.59340875
Iteration 17, loss = 0.55746448
Iteration 18,

/home/ahmed/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


'\nYou should get the following test accuracies the first time \n\nSVM accuracy ~ 97.70833333333333\nKNN accuracy ~ 96.52777777777779\nNN accuracy ~ 93.95833333333333\n'

In [35]:
run_experiment('hsv_hist')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 32.083333333333336
KNN accuracy ~ 32.708333333333336
NN accuracy ~ 9.722222222222223
"""

# Why low accuracies?
# The HSV histogram is not an ideal representation for these images.  
# It fails to capture spatial information within the image.  
# Instead, it only represents the color distribution, neglecting how colors are arranged or structured.

Loading dataset. This will take time ...
[INFO] processed 1000/7200
[INFO] processed 2000/7200
[INFO] processed 3000/7200
[INFO] processed 4000/7200
[INFO] processed 5000/7200
[INFO] processed 6000/7200
[INFO] processed 7000/7200
Finished loading dataset.
############## Training SVM ##############
SVM accuracy: 29.51388888888889 %
############## Training KNN ##############
KNN accuracy: 31.041666666666668 %
############## Training NN ##############
Iteration 1, loss = 2.19868060
Iteration 2, loss = 2.19828544
Iteration 3, loss = 2.19799452
Iteration 4, loss = 2.19776093
Iteration 5, loss = 2.19762199
Iteration 6, loss = 2.19743375
Iteration 7, loss = 2.19735425
Iteration 8, loss = 2.19723812
Iteration 9, loss = 2.19714257
Iteration 10, loss = 2.19707422
Iteration 11, loss = 2.19703091
Iteration 12, loss = 2.19697293
Iteration 13, loss = 2.19692347
Iteration 14, loss = 2.19689786
Iteration 15, loss = 2.19686126
Iteration 16, loss = 2.19682432
Iteration 17, loss = 2.19679189
Iteration 18

'\nYou should get the following test accuracies the first time \n\nSVM accuracy ~ 32.083333333333336\nKNN accuracy ~ 32.708333333333336\nNN accuracy ~ 9.722222222222223\n'

In [36]:
run_experiment('raw')
"""
You should get the following test accuracies the first time 

SVM accuracy ~ 85.06944444444444
KNN accuracy ~ 93.95833333333333
NN accuracy ~ 88.68055555555556
"""

Loading dataset. This will take time ...
[INFO] processed 1000/7200
[INFO] processed 2000/7200
[INFO] processed 3000/7200
[INFO] processed 4000/7200
[INFO] processed 5000/7200
[INFO] processed 6000/7200
[INFO] processed 7000/7200
Finished loading dataset.
############## Training SVM ##############


/home/ahmed/.local/lib/python3.10/site-packages/sklearn/svm/_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


SVM accuracy: 85.06944444444444 %
############## Training KNN ##############
KNN accuracy: 94.30555555555556 %
############## Training NN ##############
Iteration 1, loss = 9.39711486
Iteration 2, loss = 0.81859566
Iteration 3, loss = 0.51227862
Iteration 4, loss = 0.34988353
Iteration 5, loss = 0.27270159
Iteration 6, loss = 0.22819969
Iteration 7, loss = 0.18540904
Iteration 8, loss = 0.16707241
Iteration 9, loss = 0.14052266
Iteration 10, loss = 0.12797871
Iteration 11, loss = 0.11036350
Iteration 12, loss = 0.10413438
Iteration 13, loss = 0.09694497
Iteration 14, loss = 0.09015458
Iteration 15, loss = 0.08607817
Iteration 16, loss = 0.07316021
Iteration 17, loss = 0.06742490
Iteration 18, loss = 0.05711264
Iteration 19, loss = 0.04980257
Iteration 20, loss = 0.04401006
Iteration 21, loss = 0.04896514
Iteration 22, loss = 0.04736412
Iteration 23, loss = 0.04049700
Iteration 24, loss = 0.03738787
Iteration 25, loss = 0.03290233
Iteration 26, loss = 0.02894445
Iteration 27, loss = 0.0

/home/ahmed/.local/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  warnings.warn(


'\nYou should get the following test accuracies the first time \n\nSVM accuracy ~ 85.06944444444444\nKNN accuracy ~ 93.95833333333333\nNN accuracy ~ 88.68055555555556\n'

The classifiers list now has models trained on the last feature set you ran an experiment on. You can play around with it checking the probability it gives to each label, given an image.

In [59]:
# Example (test2.jpg is a 2)
labels_map = {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9}
test_img_path = r'test2.jpg'
img = cv2.imread(test_img_path)
features = extract_features(img, 'raw')  # be careful of the choice of feature set\
nn = classifiers['NN']
nn.predict_proba([features])
predict=nn.predict([features])
print("the prediction is: "+str(labels_map[predict[0]]))

# Example (test5.jpg is a 5)
test_img_path = r'test5.jpg'
img = cv2.imread(test_img_path)
features = extract_features(img, 'raw')  # be careful of the choice of feature set\
nn = classifiers['NN']
nn.predict_proba([features])
predict=nn.predict([features])
print("the prediction is: "+str(labels_map[predict[0]]))




the prediction is: 2
the prediction is: 5


Try to get a better accuracy by changing the model hyperparameters and retraining.